In [ ]:
from datetime import datetime

import qnexus as qnx

from pytket import Circuit



In [ ]:
my_project = qnx.project.get_only(name_like="VanyaTest")

qnx.context.set_active_project(my_project)

my_job_name_prefix = datetime.now()

## Compile Jobs

In [ ]:
c = Circuit(2,2).H(0).CX(0,1).measure_all()

my_circuit_ref = qnx.circuit.upload(
    circuit=c,
    name="A circuit 17 May",
)

In [ ]:
my_circuit_ref.df()

In [ ]:
compile_job_ref = qnx.compile(
    circuits=[my_circuit_ref],
    name=f"{my_job_name_prefix}_compile",
    optimisation_level=1,
    target=qnx.QuantinuumConfig(device_name="H1-1SC")
)

In [ ]:
qnx.job.wait_for(compile_job_ref)

In [ ]:
# Pretending we lost the compile_job_ref variable, we can retrieve it using filters

job_matches = qnx.job.get(name_like=f"{my_job_name_prefix}_compile")

my_job = job_matches.list()[0]

my_job.df()

In [ ]:
# get results
compile_results = qnx.job.results(compile_job_ref)

compile_results.df()

In [ ]:
compile_results[0].df()

In [ ]:
compile_results[0].get_passes().df()

In [ ]:
compile_results[0].get_output().download_circuit()

In [ ]:
passes = compile_results[0].get_passes()


passes.df()

## Execute Jobs

In [ ]:
compiled_circuits = [item.get_output() for item in compile_results]

In [ ]:
compiled_circuits

In [ ]:
execute_job_ref = qnx.execute(
    circuits=compiled_circuits,
    name=f"{my_job_name_prefix}1_execute",
    n_shots=[100]* len(compiled_circuits),
    target=qnx.QuantinuumConfig(device_name="H1-1SC"),
)

In [ ]:
qnx.job.wait_for(execute_job_ref)

In [ ]:
execute_results = qnx.job.results(execute_job_ref)

execute_results.df()

In [ ]:
execute_results[0].download_backend_info()

In [ ]:
execute_results[0].download_result()

In [ ]:
# Example of a job erroring

In [ ]:
execute_job_ref = qnx.execute(
    circuits=[my_circuit_ref],
    name=f"error_execute_{datetime.now()}",
    n_shots=[100],
    target=qnx.QuantinuumConfig(device_name="H1-1LE")
)

In [ ]:
qnx.job.wait_for(execute_job_ref)